# Deploy SciKit-Learn Model to Production using Panini in 3 minutes
###### https://panini.ai/
###### Panini is a platform that serves ML/DL models at low latency and makes the ML model deployment to production from a few days to a few minutes. 

Once deployed in Panini’s server, it will provide you with an API key to infer the model. Panini query engine is developed in C++, which provides very low latency during model inference and Kubernetes cluster is being used to store the model so, it is scalable to multiple nodes. Panini also takes care of caching and batching inputs during model inference.

#### Let's deploy the classic regression to Classify different flowers using the Iris dataset

This data sets consists of 3 different types of irises’ (Setosa, Versicolour, and Virginica) petal and sepal length, stored in a 150x4 numpy.ndarray

https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html

In [185]:
#Train the Model
from sklearn import linear_model, datasets
from sklearn import datasets
import numpy as np
import pickle
iris = datasets.load_iris()
X = iris.data[:, :] 
Y = iris.target
model = linear_model.LogisticRegression(C=1e5)
model.fit(X,Y)

#Once the model is trained, save it using pickle.
filename = 'model.pkl'
outfile = open(filename,'wb')
pickle.dump(model,outfile)
outfile.close()

#Load the model back, just to make sure everything is good!
infile = open(filename,'rb')
model = pickle.load(infile)
infile.close()
model

/Users/avin/anaconda3/envs/clipper/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/avin/anaconda3/envs/clipper/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

### You need to upload at least 3 files to panini:
 - predict.py
 - requirements.txt
 - model.pth¶
 
### predict.py should have two functions:

 - def load(path): path -> saved state_dict has to be stored as .pth
     returns model.
     
     load function is used to load the model. For our demo, we saved the model as "model.pkl"
     Once the model is loaded, it will return back the model.
     
     
 - def predict(input_from_user): -> inputs_from_user is a list[]
     returns prediction as a list
     
     This function gets executed, when a user sends POST request to our API. User's data is stored in the variable "input_from_client". It will return the prediction.

### requirements.txt
Create a file called "requirements.txt" and write these two entries!

scikit-learn

numpy

In [186]:
#predict.py

def load(path):
    import pickle
    infile = open(path,'rb')
    model = pickle.load(infile)
    infile.close()
    return model


#Main predict() function!
def predict(input_from_client):
    #import sklearn
    from sklearn import linear_model, datasets

    from numpy import array
    
    #load our model. We don't need load() function but it's nice to have. 
    model = load("model.pkl")
    
    #Once the model is loaded, feed the client's data into our model
    prediction = model.predict(input_from_client)
    value = []
    for label in prediction:
        if label == 0:
            value.append('Setosa')
        elif label == 1:
            value.append('Virginica')
        else:
            value.append('Versicolour')
    #Return prediction back to the client.
    return value

In [181]:
predict([X[0], X[101]])

['Setosa', 'Versicolour']

### Upload to panini.ai
 - Sign into https://panini.ai/
 

 - Give your model a name.
 

 - Choose input as floats
 

 - Upload: predict.py, requirements.txt and model.pkl
 

 - Do not upload anything for extrafiles (optional)


 - Press Deploy!
 

 - Refresh the page, and wait for API URL to appear

### Infer using Panini.ai

In [192]:
#Send POST request to API URL
import requests, json

#our API_URL
API_LINK = "https://api.panini.ai/jm0gox1kqty3xjni83fxahcf6ks2-scikit7/predict"

#Send Sepal Length, Sepal Width, Petal Length and Petal Width as input to classify.
data_to_send = [5.1, 3.5, 1.4, 0.2]
response = requests.post(
     API_LINK,
     headers={"Content-type": "application/json"},
     data=json.dumps({
         'input': data_to_send,
     }))
result = response.json()
result

{'query_id': 51, 'output': 'Setosa', 'default': False}

In [193]:
data

'{"input": [5.1, 3.5, 1.4, 0.2]}'

## 